In [1]:
from diffusers import LDMSuperResolutionPipeline
from diffusers.pipelines.latent_diffusion.pipeline_latent_diffusion_superresolution import preprocess

from dataset import CoverDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from models import get_models
import albumentations as A
import torch
from defs import *
from diffusers import DDPMScheduler, DDPMPipeline
from evaluate import evaluate, make_grid
import numpy as np


/root/anaconda3/envs/album/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
## Define models

In [ ]:
DEVICE = 'cuda:2'
SEED = 69


In [ ]:
model = get_models(IMSIZE).to(DEVICE)
model = model.from_pretrained('results/weights/unet')
model.sample_size = 64
model = model.to(DEVICE)
model.eval()
noise_scheduler = DDPMScheduler(num_train_timesteps=NUM_TRAIN_TIMESTEPS)

pipeline = DDPMPipeline(unet=model, scheduler=noise_scheduler)
generator = torch.Generator(device=DEVICE)
generator = generator.manual_seed(SEED)
images = pipeline(
    batch_size=64,
    generator=generator,
).images

In [ ]:
rescaler_id = "CompVis/ldm-super-resolution-4x-openimages"
rescaler = LDMSuperResolutionPipeline.from_pretrained(rescaler_id)
rescaler = rescaler.to(DEVICE)

In [2]:
DEVICE = 'cuda:2'
model = get_models(IMSIZE).to(DEVICE)
model = model.from_pretrained('results/weights/unet')
model.sample_size = 64
model = model.to(DEVICE)
model.eval()
noise_scheduler = DDPMScheduler(num_train_timesteps=NUM_TRAIN_TIMESTEPS)

In [20]:
pipeline = DDPMPipeline(unet=model, scheduler=noise_scheduler)
generator = torch.Generator(device=DEVICE)
generator = generator.manual_seed(SEED)
images = pipeline(
    batch_size=64,
    generator=generator,
).images

images = [preprocess(i) for i in images]
images = torch.cat(images)
upscaled_images = rescaler(images, num_inference_steps=100, eta=1)

100%|██████████| 100/100 [00:11<00:00,  8.76it/s]


In [21]:
grid = make_grid(upscaled_images.images, 8, 8)